# Optimize memory for API reqests #

In a scenario where you perform multiple API requests in *rapid succession,* you might be required to actively clean up memory, in order to avoid running out of memory.

Python is great at garbage collection.

However, in certain scenarios, such as when you execute thousands of API requests per second in a single RAM space, you might need to perform active garbage collection in your program.

**Techniques to actively clean up memory:**

1. Use context manager:[`with`](#1)


2. Persist to storage: [`json.dump`](#2a), [`f.write`](#2b)


3. Scope lifespan of variables as tightly as possible: [Use functions](#3)


4. Explicitly delete large objects: [`del`](#4)


5. Run garbage collection: [`gc.collect()`](#5)

## Import Libraries + Initialize Key Variables + Perform Setups ##

In [1]:
!pip install langdetect -q

import gc
import json
import os
import pprint
import requests

from langdetect import detect

countries_all = 'https://restcountries.com/v3.1/all'

target_file_name = 'countries.json'
target_file_name_2 = 'countries_2.json'

target_directory_path = os.getenv('DOWNLOADS_PATH')

# Create target directory.
if not(os.path.exists(target_directory_path)):
  os.makedirs(target_directory_path)

target_file_path = os.path.join(target_directory_path, target_file_name)

target_file_path_2 = os.path.join(target_directory_path, target_file_name_2)

request_success_message = 'Request was successful: Received Success 200 Status Code.'
request_not_success_message = 'Request was not successful.'

#### Create function to detect whether text is English. ####

In [2]:
def is_english(text):

  if detect(text) == 'en':
    return True
  else:
    return False

In [3]:
text = 'República Federativa do Brasil'
print(f'{text} is English: {is_english(text)}\n')

text = 'Federative Republic of Brazil'
print(f'{text} is English: {is_english(text)}\n')

text = 'Republic of Moldova'
print(f'{text} is English: {is_english(text)}\n')

text = 'Republica Moldova'
print(f'{text} is English: {is_english(text)}\n')

text = 'Hashemite Kingdom of Jordan'
print(f'{text} is English: {is_english(text)}\n')

text = 'al-Mamlakah al-Urdunīyah al-Hāshimīyah'
print(f'{text} is English: {is_english(text)}\n')

text = 'United States of America'
print(f'{text} is English: {is_english(text)}\n')

República Federativa do Brasil is English: False

Federative Republic of Brazil is English: True

Republic of Moldova is English: True

Republica Moldova is English: False

Hashemite Kingdom of Jordan is English: True

al-Mamlakah al-Urdunīyah al-Hāshimīyah is English: False

United States of America is English: True



## GET API request ##

In [4]:
countries_response = requests.get('https://restcountries.com/v3.1/all')

if countries_response.status_code == 200:
  print(request_success_message)
else:
  print(request_not_success_message)

Request was successful: Received Success 200 Status Code.


In [5]:
# Print type of response JSON.
print(type(countries_response.json()))

# Print length of response JSON.
print(len(countries_response.json()))

<class 'list'>
250


In [6]:
# Print second element of response JSON: USA
pprint.pprint(countries_response.json()[10])

{'altSpellings': ['NU'],
 'area': 260.0,
 'capital': ['Alofi'],
 'capitalInfo': {'latlng': [-19.02, -169.92]},
 'car': {'side': 'left', 'signs': ['NZ']},
 'cca2': 'NU',
 'cca3': 'NIU',
 'ccn3': '570',
 'coatOfArms': {},
 'continents': ['Oceania'],
 'currencies': {'NZD': {'name': 'New Zealand dollar', 'symbol': '$'}},
 'demonyms': {'eng': {'f': 'Niuean', 'm': 'Niuean'},
              'fra': {'f': 'Niuéenne', 'm': 'Niuéen'}},
 'flag': '🇳🇺',
 'flags': {'png': 'https://flagcdn.com/w320/nu.png',
           'svg': 'https://flagcdn.com/nu.svg'},
 'idd': {'root': '+6', 'suffixes': ['83']},
 'independent': False,
 'landlocked': False,
 'languages': {'eng': 'English', 'niu': 'Niuean'},
 'latlng': [-19.03333333, -169.86666666],
 'maps': {'googleMaps': 'https://goo.gl/maps/xFgdzs3E55Rk1y8P9',
          'openStreetMaps': 'https://www.openstreetmap.org/relation/1558556'},
 'name': {'common': 'Niue',
          'nativeName': {'eng': {'common': 'Niue', 'official': 'Niue'},
                         'niu

In [7]:
# Print capital of the first element.
countries_response.json()[1].get('capital')[0]

'Washington, D.C.'

In [8]:
# Print type of response text.
print(type(countries_response.text))

# Print length of response text.
print(len(countries_response.text))

# Print 1st 250 characters of response text.
print(countries_response.text[:250])

<class 'str'>
776703
[{"name":{"common":"Moldova","official":"Republic of Moldova","nativeName":{"ron":{"official":"Republica Moldova","common":"Moldova"}}},"tld":[".md"],"cca2":"MD","ccn3":"498","cca3":"MDA","cioc":"MDA","independent":true,"status":"officially-assigned"


In [9]:
# Convert response text to JSON object.
countries_response_json_2 = json.loads(countries_response.text)

In [10]:
# Print type of response JSON.
print(type(countries_response_json_2))

# Print length of response JSON.
print(len(countries_response_json_2))

<class 'list'>
250


## Active Memory Optimization ##

<a id='1'></a>
### 1. Use context manager: `with` ###

In [11]:
with requests.get(countries_all) as countries_response_3:
  countries_response_json_3 = countries_response_3.json()

  if countries_response_3.status_code == 200:
    print(request_success_message)

    if len(countries_response_json_3) > 1:
      element_1_capital = countries_response_json_3[1].get('capital')[0]
      print(f'Capital of Element 1: {element_1_capital}')

  else:
    print(request_not_success_message)

Request was successful: Received Success 200 Status Code.
Capital of Element 1: Washington, D.C.


<a id='2a'></a>
### 2a. Persist to storage ASAP: `json.dump()` ###

In [12]:
# Save JSON to file.
with open(target_file_path, 'w') as f:
  json.dump(countries_response_json_2, f)
  print(f'Successfully saved {target_file_name}.')

Successfully saved countries.json.


<a id='2b'></a>
### 2b. Persist to storage ASAP: `f.write()` ###

In [13]:
countries_response_str_2 = json.dumps(countries_response_json_2)

with open(target_file_path_2, 'w') as f:
  f.write(countries_response_str_2)
  print(f'Successfully saved {target_file_name_2}.')

Successfully saved countries_2.json.


<a id='3'></a>
### 3. Scope lifespan of variables as tightly as possible: Use functions ###

Use function scopes to limit the lifespan of large objects. Objects created within a function are typically cleaned up when the function exits.

#### Create function to get country name and capital. ####

In [14]:
def get_country_name_capital(countries_endpoint_url, element_ids):

  request_success_message = 'Request was successful: Received Success 200 Status Code.\n'
  request_not_success_message = 'Request was not successful.\n'
  country = None

  with requests.get(countries_endpoint_url) as countries_response:
    countries_response_json = countries_response.json()

    if countries_response.status_code == 200:
      print(request_success_message)

      for element_id in element_ids:

        country_names = countries_response_json[element_id].get('altSpellings')

        for country_name in country_names:

          if is_english(country_name):
            
              country = country_name

              print(f'Country of Element {element_id}: {country}')

              break

        capital = countries_response_json[element_id].get('capital')[0]
        
        print(f'Capital of Element {element_id}: {capital}\n')

    else:
      print(request_not_success_message)

In [15]:
countries_endpoint_url = 'https://restcountries.com/v3.1/all'

get_country_name_capital(countries_endpoint_url, [0, 1, 5, 20, 25])

Request was successful: Received Success 200 Status Code.

Country of Element 0: Republic of Moldova
Capital of Element 0: Chișinău

Country of Element 1: United States of America
Capital of Element 1: Washington, D.C.

Country of Element 5: Federative Republic of Brazil
Capital of Element 5: Brasília

Country of Element 20: Republic of Madagascar
Capital of Element 20: Antananarivo

Country of Element 25: Republic of Paraguay
Capital of Element 25: Asunción



<a id='4'></a>
### 4. Explicitly delete large objects: `del` ###

In [16]:
if 'countries_response' in locals() or 'countries_response' in globals():
    del countries_response

if 'countries_response_json_2' in locals() or 'countries_response_json_2' in globals():
    del countries_response_json_2

if 'countries_response_json_3' in locals() or 'countries_response_json_3' in globals():
    del countries_response_json_3

if 'countries_response_str_2' in locals() or 'countries_response_str_2' in globals():
    del countries_response_str_2

if 'countries_response_3' in locals() or 'countries_response_3' in globals():
    del countries_response_3

<a id='5'></a>
### 5. Run garbage collection: `gc.collect()` ###

In [17]:
gc.collect()

0